In [1]:
import os
import pandas as pd
import glob
from pyproj import Geod
import numpy as np

In [2]:
usbl = glob.glob('S:\HSV\Log Files\clean\*.csv')

In [9]:
usbl = pd.concat([pd.read_csv(ub,index_col='timestamp',parse_dates=['timestamp']) for ub in usbl])

In [10]:
def processs_keys(item):
    return {'timestamp':item.timestamp,'keytype':item['data']['keytype'],'keyname':item['data']['keyname']}

def process_data(file):
    data = pd.read_json(file,lines=True)
    keys = pd.DataFrame(list(data[data.message.str.endswith('keystroke.eng')].apply(processs_keys,axis=1)))
    keys.set_index('timestamp',inplace=True)
    keys =pd.pivot_table(keys, values='keyname', index=['timestamp'],columns=['keytype'],aggfunc='first')
    benthos =pd.concat([keys['benthic'].resample('1S').first().ffill(),keys['substrate'].resample('1S').first().ffill()], axis=1)
    cols = ['depth_meters','longitude','latitude']
    benthos =benthos.join(usbl[cols])
    benthos[cols] = benthos[cols].interpolate()
    benthos = benthos[~benthos.longitude.isna()]
    geod = Geod(ellps='WGS84')
    azimuth1, azimuth2, distance = geod.inv(benthos.longitude.values[:-1], benthos.latitude.values[:-1], benthos.longitude.values[1:], benthos.latitude.values[1:])
    benthos['distance'] =np.append(distance,0)
    benthos =benthos.join(keys.loc[~keys.animal.isna(),'animal'],how='outer').sort_index()
    benthos['benthic'] = benthos['benthic'].ffill()
    benthos['substrate'] = benthos['substrate'].ffill()
    benthos[cols] = benthos[cols].ffill()
    benthos.loc[benthos.distance.isna(),'distance'] =0
    benthos['station']=station =int(file.split('_')[7])
    benthos.index.name = 'timestamp'
    return benthos

In [11]:
import pandas as pd
import glob

files = glob.glob('S:/HSV/Tappity/*.json')
usbl = glob.glob('S:\HSV\Log Files\clean\*.csv')
usbl = pd.concat([pd.read_csv(ub,index_col='timestamp',parse_dates=['timestamp']) for ub in usbl])
ub = usbl.resample('1S').first()

def process_data(file):
    data = pd.read_json(file,lines=True)
    keys = pd.DataFrame(list(data[data.message.str.endswith('keystroke.eng')].apply(processs_keys,axis=1)))
    keys.set_index('timestamp',inplace=True)
    keys =pd.pivot_table(keys, values='keyname', index=['timestamp'],columns=['keytype'],aggfunc='first')
    benthos =pd.concat([keys['benthic'].resample('1S').first().ffill(),keys['substrate'].resample('1S').first().ffill()], axis=1)
    benthos['station']=station =int(file.split('_')[2])
    benthos.index.name = 'timestamp'
    return benthos


for file in files:
    print(file)
    data = process_data(file)
    x =data.join(ub).interpolate()
    x.to_csv(file.replace('.json','.csv'),index=True)

S:/HSV/Tappity\TAPPITY_NDR2019_2601_2019_03_26_03_48_21.json
S:/HSV/Tappity\TAPPITY_NDR2019_2803_2019_03_28_06_21_28.json
S:/HSV/Tappity\TAPPITY_NDR2019_2812_2019_03_28_07_01_14.json
S:/HSV/Tappity\TAPPITY_NDR2019_2818_2019_03_28_04_57_47.json
S:/HSV/Tappity\TAPPITY_NDR2019_2818_2019_03_28_04_59_15.json
S:/HSV/Tappity\TAPPITY_NDR2019_2904_2019_03_29_04_23_26.json
S:/HSV/Tappity\TAPPITY_NDR2019_2905_2019_03_29_00_33_11.json
S:/HSV/Tappity\TAPPITY_NDR2019_2906_2019_03_29_03_23_41.json
S:/HSV/Tappity\TAPPITY_NDR2019_2907_2019_03_29_01_06_32.json
S:/HSV/Tappity\TAPPITY_NDR2019_2909_2019_03_29_02_00_22.json
S:/HSV/Tappity\TAPPITY_NDR2019_2910_2019_03_29_02_43_48.json
S:/HSV/Tappity\TAPPITY_NDR2019_3014_2019_03_29_23_37_00.json
S:/HSV/Tappity\TAPPITY_NDR2019_3015_2019_03_30_00_20_30.json
S:/HSV/Tappity\TAPPITY_NDR2019_3016_2019_03_30_03_52_04.json
S:/HSV/Tappity\TAPPITY_NDR2019_3017_2019_03_30_03_16_12.json
S:/HSV/Tappity\TAPPITY_NDR2019_3019_2019_03_30_01_11_37.json
S:/HSV/Tappity\TAPPITY_N